In [ ]:
import cv2
from vtk.util import numpy_support as nps
import vtk
import os
import numpy as np
from tvtk.api import tvtk, write_data # pip install mayavi

# settings
folder_png = "images/mask_png"
folder_vtk = "images/mask_vtk"

# create folder
os.makedirs(folder_vtk, exist_ok=True)

# get images
images = [os.path.join(folder_png, file) for file in os.listdir(folder_png)]

def img_to_points(image_path):
    # get image
    image = cv2.imread(image_path)[:,:,0]
    
    # to mask
    ret, image_mask = cv2.threshold(image,0,255,cv2.THRESH_BINARY)

    # get contours
    contours, hierarchy = cv2.findContours(image_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # keep only biggest contour (in case it is separated for some reason)
    contours = max(contours, key = cv2.contourArea)
    
    # make polygon from contour
    polygons = np.array(contours).squeeze()
    num_points = len(polygons)

    # convert to 3d dimension by setting z to 0
    polygons_3d = np.zeros((num_points, 3))
    polygons_3d[:,:2] = polygons
    
    # create lines
    num_points = num_points - 1
    lines = np.zeros((num_points, 2), 'l')
    lines[:, 0] = np.arange(0, num_points - 0.5, 1, 'l')
    lines[:, 1] = np.arange(1, num_points + 0.5, 1, 'l')
    
    return polygons_3d, lines

# get polygon
for image in images:
    mesh_points, mesh_lines = img_to_points(image)
    mesh = tvtk.PolyData(points=mesh_points, lines=mesh_lines)
    path_out = image.replace(folder_png, folder_vtk).replace(".png", ".vtk")
    write_data(mesh, path_out)
    
print("vtks created succesfully")